In [1]:
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow import keras
import numpy as np
import sys
import matplotlib.pyplot as plt
import time

# using float64 as default for each layer
policy = tf.keras.mixed_precision.Policy("float64")
tf.keras.mixed_precision.set_global_policy(policy)

### Define Gaussian Layer

In [2]:
# kernal function
class Gaussian(layers.Layer):
  def __init__(self, Nx=25, Ny=25, Lx=1.0, Ly=1.0, alpha=1/8, sigma=None):
    super().__init__()

    dx = Lx / Nx; dy = Ly / Ny;

    if sigma==None:
      sigma = alpha * (dx**2+dy**2)**0.5

    x_c = tf.linspace(dx/2, Lx-dx/2, Nx)
    y_c = tf.linspace(dy/2, Ly-dy/2, Ny)

    x_mg, y_mg = tf.meshgrid(x_c, y_c)

    self.x_mg = tf.reshape(tf.cast(x_mg, tf.float64),[1,1,-1])
    self.y_mg = tf.reshape(tf.cast(y_mg, tf.float64),[1,1,-1])
    self.sigma = sigma

  def call(self, Source_XY, Source_Q):
    Nb = tf.shape(Source_XY)[0]

    x_src = Source_XY[:,:,0:1]
    y_src = Source_XY[:,:,1:2]
    q_src = Source_Q

    D_square = tf.square(self.x_mg-x_src) + tf.square(self.y_mg-y_src)

    gauss = tf.exp(-0.5 * D_square / self.sigma**2 )

    b_star = gauss / tf.reduce_sum(gauss,axis=2,keepdims=True)
    b_star *= q_src
    b_star = tf.reduce_sum(b_star, axis=1)

    return b_star

### Define Poisson Layer

In [3]:
class Poisson(layers.Layer):
  def __init__(self, Lx=1, Ly=1, Nx=25, Ny=25, Perm=None, viscosity=0.001, Ns=2):
    super().__init__()

    if Perm == None:
      K_mu = tf.ones([Ny,Nx], tf.float64) * 200 * 1E-15 / viscosity
    else:
      K_mu = Perm / viscosity

    self.Nx = Nx
    self.Ny = Ny
    self.N = Nx * Ny
    self.dx = Lx / Nx
    self.dy = Ly / Ny
    self.Ns = Ns
    self.Lx = Lx
    self.Ly = Ly

    paddings = tf.constant([[1, 1], [1, 1]])
    K = tf.pad(K_mu, paddings, "CONSTANT")

    self.Tim = (self.dy/self.dx)*2*K[1:-1, 1:-1]*K[1:-1, :-2]/(K[1:-1, 1:-1]+K[1:-1, :-2])
    self.Tip = (self.dy/self.dx)*2*K[1:-1, 1:-1]*K[1:-1, 2:]/(K[1:-1, 1:-1]+K[1:-1, 2:])
    self.Tjm = (self.dx/self.dy)*2*K[1:-1, 1:-1]*K[:-2, 1:-1]/(K[1:-1, 1:-1]+K[:-2, 1:-1])
    self.Tjp = (self.dx/self.dy)*2*K[1:-1, 1:-1]*K[2:, 1:-1]/(K[1:-1, 1:-1]+K[2:, 1:-1])

    self.A = tf.zeros([self.N, self.N], tf.float64)
    self.A = tf.linalg.set_diag(self.A, tf.reshape(self.Tim, [-1])[1:], k=-1)
    self.A = tf.linalg.set_diag(self.A, tf.reshape(self.Tip, [-1])[:-1], k=1)
    self.A = tf.linalg.set_diag(self.A, tf.reshape(self.Tjp, [-1])[:self.N-Nx], k=Nx)
    self.A = tf.linalg.set_diag(self.A, tf.reshape(self.Tjm, [-1])[Nx:], k=-Nx)

    # Apply External Boundary Conditions - Constant Pressures at Left (1) and Right (0)
    BCID_i = tf.concat([tf.zeros([Ny,1],tf.int32),tf.ones([Ny,1],tf.int32)*(Nx-1)], axis=0)
    BCID_j = tf.reshape(tf.concat([tf.range(Ny),tf.range(Ny)], axis=0), [-1,1])
    BCIDs  = tf.concat([BCID_j,BCID_i], axis=1)

    T_bc_im = 2*(self.dy/self.dx)*K[1:-1, 1]
    T_bc_ip = 2*(self.dy/self.dx)*K[1:-1,-2]
    BCVal = tf.concat([T_bc_im, T_bc_ip], axis=0)

    self.Tim = tf.tensor_scatter_nd_update(self.Tim, BCIDs[:Ny,:], BCVal[:Ny])
    self.Tip = tf.tensor_scatter_nd_update(self.Tip, BCIDs[Ny:,:], BCVal[Ny:])
    ###################################################################################

    self.A = tf.linalg.set_diag(self.A, -tf.reshape(self.Tim+self.Tip+self.Tjp+self.Tjm, [-1]), k=0)

    self.b = tf.zeros([self.N,1], tf.float64)

    # Apply External Boundary Conditions - Constant Pressures at Left (1E+6) and Right (0)
    self.Pbc_L = 1E+6
    self.Pbc_R = 0
    b_Val = -tf.concat([self.Tim[:,0]*self.Pbc_L, self.Tip[:,-1]*self.Pbc_R], axis=0)
    b_IDs = BCIDs[:,0:1]*Nx+BCIDs[:,1:2]
    self.b = tf.tensor_scatter_nd_add(self.b,
                                      tf.concat([b_IDs, tf.zeros(b_IDs.shape,tf.int32)], axis=1),
                                      b_Val)
    ###################################################################################

    self.A = tf.expand_dims(self.A, axis=0)
    self.b = tf.expand_dims(self.b, axis=0)

  def call(self, b_star):
    Nb = tf.shape(b_star)[0]

    # For Neumann BC - Constant Source Strength
    A = tf.tile(self.A, [Nb,1,1])
    b = tf.tile(self.b, [Nb,1,1])
    b += tf.reshape(b_star, [Nb,-1,1] )
    #############################################

    pressure = tf.linalg.solve(A,b)

    return pressure

### Source Localization using Poisson Layer

#### Produce Target Pressure Field

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf

np.random.seed(10)

Nx=25; Ny=25

phi_ref = 0.2
sigma = [0.01, 0.03, 0.05]

phi_001 = np.random.normal(phi_ref, sigma[0], Nx*Ny)
phi_003 = np.random.normal(phi_ref, sigma[1], Nx*Ny)
phi_005 = np.random.normal(phi_ref, sigma[2], Nx*Ny)

fig, axs = plt.subplots(2, 3, figsize=(6,4), sharey=True, tight_layout=True)

axs[0,0].hist(phi_001, bins=200)
axs[0,1].hist(phi_003, bins=200)
axs[0,2].hist(phi_005, bins=200)
axs[0,0].set_title('(a) $\phi$ ($\sigma=0.01$)', fontsize=12, y=-0.5)
axs[0,0].set_xlim([0.15, 0.25])
axs[0,0].set_ylim([0, 20])
axs[0,1].set_title('(b) $\phi$ ($\sigma=0.03$)', fontsize=12, y=-0.5)
axs[0,1].set_xlim([0.1, 0.3])
axs[0,2].set_title('(c) $\phi$ ($\sigma=0.05$)', fontsize=12, y=-0.5)
axs[0,2].set_xlim([0, 0.4])

k_001 = 200*phi_001/phi_ref*(phi_001*(1-phi_ref)/phi_ref/(1-phi_001))**2
k_003 = 200*phi_003/phi_ref*(phi_003*(1-phi_ref)/phi_ref/(1-phi_003))**2
k_005 = 200*phi_005/phi_ref*(phi_005*(1-phi_ref)/phi_ref/(1-phi_005))**2

axs[1,0].hist(k_001, bins=200)
axs[1,1].hist(k_003, bins=200)
axs[1,2].hist(k_005, bins=200)
axs[1,0].set_title('(d) $k$ ($\sigma=0.01$)', fontsize=12, y=-0.5)
axs[1,0].set_xlim([100, 300])
axs[1,1].set_title('(e) $k$ ($\sigma=0.03$)', fontsize=12, y=-0.5)
axs[1,1].set_xlim([0, 800])
axs[1,1].set_xticks([0, 250, 500])
axs[1,2].set_title('(f) $k$ ($\sigma=0.05$)', fontsize=12, y=-0.5)
axs[1,2].set_xlim([0, 1500])
fig.tight_layout()
plt.show()

In [ ]:
# Produce Target Pressure Field
Nb=1; Ns=1; eps = 0.0001; Nx=25; Ny=25; Lx=1.0; Ly=1.0
alpha = 1/8

########## Generate a Permeability Field #########
# Homogeneous Domain
Perm_Homo = tf.ones([Ny,Nx], tf.float64) * 200 * 1E-15
# Heterogeneous Domain
Perm_Hete = tf.constant(k_005.reshape([Ny,Nx]), tf.float64) * 1E-15
##################################################

random_source = False
if random_source:
  # Randomly Generated Source Locations
  x = tf.random.uniform(shape=[Nb,Ns,1], minval=eps, maxval=1-eps, dtype=tf.float64)
  y = tf.random.uniform(shape=[Nb,Ns,1], minval=eps, maxval=1-eps, dtype=tf.float64)
  SRC_XY = tf.concat([x,y], axis=2)
  #####################################
else:
  # Customized Source Locations
  one = tf.ones([Nb,1,1], tf.float64)
  source_point_1 = tf.concat([0.05*one,0.05*one], axis=2)
  source_point_2 = tf.concat([0.75*one,0.75*one], axis=2)
  source_point_3 = tf.concat([0.25*one,0.75*one], axis=2)
  source_point_4 = tf.concat([0.75*one,0.25*one], axis=2)
  source_point_5 = tf.concat([0.5*one,0.5*one], axis=2)
  SRC_XY = source_point_1
  Ns = SRC_XY.shape[1]
  ###############################################

print ("True Source Location:", SRC_XY.numpy())

SRC_Q = tf.ones([Nb,Ns,1], tf.float64) * (-1E-4)

b_star = Gaussian(Nx=Nx, Ny=Ny, alpha=alpha)(SRC_XY, SRC_Q)
P_target = Poisson(Nx=Nx, Ny=Ny, Perm=Perm_Homo)(b_star)

# Image Plot
fig, axs = plt.subplots(1, 2)
axs[0].imshow(tf.reshape(b_star, [Nb,Ny,Nx])[0],cmap='gray',interpolation='nearest')
axs[1].axis("off")
axs[1].imshow(tf.reshape(P_target,[Nb,Ny,Nx])[0],cmap='jet',interpolation='spline16')
plt.show()

#### Initial Guess - Source Locations

In [ ]:
# Generate Start Points for Source Localizations
def generate_Initial_Sources(N_node_x, N_node_y):
  x = tf.reshape(tf.linspace(Lx/N_node_x/2, 1-Lx/N_node_x/2, N_node_x), [-1,1])
  y = tf.reshape(tf.linspace(Ly/N_node_y/2, 1-Ly/N_node_y/2, N_node_y), [-1,1])

  x = tf.cast(x, tf.float64)
  y = tf.cast(y, tf.float64)

  x = tf.tile(x, [N_node_y,1])
  y = tf.reshape(tf.tile(y, [1,N_node_x]), [N_node_x*N_node_y,1])

  init_guess = tf.reshape(tf.concat([x,y], axis=1), [1,-1, 2])

  return init_guess

# set number of points to start localization
num_inital_guess = 1

one = tf.ones([1,1,1], tf.float64)
if num_inital_guess == 1:
  init_guess = tf.concat([0.5*one,0.5*one], axis=2)
elif num_inital_guess == 5:
  init_guess1 = tf.concat([0.25*one,0.25*one], axis=2)
  init_guess2 = tf.concat([0.75*one,0.75*one], axis=2)
  init_guess3 = tf.concat([0.25*one,0.75*one], axis=2)
  init_guess4 = tf.concat([0.75*one,0.25*one], axis=2)
  init_guess5 = tf.concat([0.5*one,0.5*one], axis=2)
  init_guess = tf.concat([init_guess1,init_guess2,init_guess3,init_guess4,init_guess5], axis=1)
else:
  num_points = int(num_inital_guess**0.5+0.5)
  init_guess = generate_Initial_Sources(num_points,num_points)

print (init_guess)

# Initialize src_q based on the number of initial guesses
src_q = tf.ones([init_guess.shape[0],init_guess.shape[1],1], tf.float64) * (-1E-4)

#### Define Function - Mean Euclidean Distance

In [7]:
# Define Euclidian Distance as the metrics
def mean_euclidean_distance(true_xy, pred_xy):
  pred_center_xy = tf.reduce_mean(pred_xy, axis=1)
  true_center_xy = tf.reduce_mean(true_xy, axis=1)

  med = tf.square(pred_center_xy-true_center_xy)
  med = tf.reduce_sum(med, axis=1)

  return tf.sqrt(med)

#### Define Function - Plot Partial Pressure Field

In [8]:
def plot_partial_pressure(p, sample, nx, ny, frac):
  Psample = tf.reshape(sample[:int(frac*nx*ny)], [-1,1])
  row = Psample[:int(frac*nx*ny)] // nx
  col = Psample[:int(frac*nx*ny)] % nx
  ids = tf.concat([row,col], axis=1)

  P_partial = tf.gather(p[0,:,0], sample[:int(frac*nx*ny)])

  scatter = tf.scatter_nd(ids, P_partial, [ny,nx])

  vmin = tf.reduce_min(p[0,:,0])
  vmax = tf.reduce_max(p[0,:,0])

  fig, axs = plt.subplots(1, 2)
  axs[0].axis("off")
  axs[0].imshow(tf.reshape(p,[1,Ny,Nx])[0], cmap='jet')
  axs[1].axis("off")
  axs[1].imshow(tf.reshape(scatter,[1,Ny,Nx])[0], vmin=vmin, vmax=vmax, cmap='jet')
  plt.show()

#### Source Localization
Case I - Single Source
1.   In Produce Target Pressure Module, set Ns=1
2.   In Initial Guess Module, set num_inital_guess=1
3.   In Source Localization Module, set data_frac = 1

Case II - Multiple Sources
1.   In Produce Target Pressure Module, set Ns=(larger than 1)
2.   In Initial Guess Module, set num_inital_guess=(larger than 1)
3.   In Source Localization Module, set data_frac = 1

Case III - Partial Data
1.   Define The Plot Function
2.   In Source Localization Module, set data_frac = (between 0 and 1)

In [9]:
def add_src_list (src_ids, drop_ids, src_array, src_loc):
  src_loc_flat = src_loc.reshape(-1)

  new_row = np.empty((1,src_array.shape[1]))*np.nan

  if (drop_ids.shape[0]>0):
    src_ids = np.delete(src_ids, drop_ids)

  src_ids_x = src_ids * 2
  src_ids_y = src_ids * 2 + 1
  src_ids_xy = np.concatenate((src_ids_x.reshape(-1,1), src_ids_y.reshape(-1,1)), axis=1)
  src_ids_xy = src_ids_xy.reshape(-1)

  new_row[0,src_ids_xy] = src_loc_flat

  src_array = np.concatenate((src_array,new_row), axis=0)

  return src_ids, src_array

In [ ]:
alpha = 1/8
gk = Gaussian(Nx=Nx, Ny=Ny, Lx=Lx, Ly=Ly, alpha=alpha)
poisson = Poisson(Nx=Nx, Ny=Ny, Lx=Lx, Ly=Ly, Ns=Ns, Perm=Perm_Homo)

epoch = 300

all_src = []
all_loss = []
all_med = []
all_iter = []
best_src = []
best_loss = []
best_med = []
best_iter = []
all_src_np = np.array([]).reshape(0,init_guess.shape[1]*init_guess.shape[2])

drop_ids = np.array([])
src_ids = np.arange(init_guess.shape[1])

src_loc = init_guess
src_q = tf.ones([init_guess.shape[0],init_guess.shape[1],1], tf.float64) * (-1E-4)

one = tf.ones([Nb,init_guess.shape[1],1], tf.float64)
D_diag = (Lx**2+Ly**2)**0.5
max_step_size = D_diag/100*one
min_step_size = D_diag/1000*one

samples = tf.range(0, Nx*Ny)
samples = tf.random.shuffle(samples)
data_frac = 1

counter = 0
d_src_loc = 0

plot_partial_pressure(P_target, samples, Nx, Ny, data_frac)

for i in range (epoch):
  with tf.GradientTape(persistent=True, watch_accessed_variables=False) as tape:
    tape.watch(src_loc)
    source_field = gk(src_loc, src_q)
    P_pred = poisson(source_field)

    SE = tf.reshape(tf.square(P_pred-P_target),[-1])
    loss = tf.reduce_mean(tf.gather(SE, samples[:int(data_frac*Nx*Ny)]))
    med = mean_euclidean_distance(SRC_XY, src_loc)

  if (i%1==0 and src_loc.shape[1]==1):
    print (i, "\t",
           "{:.5f}".format(src_loc.numpy()[0,0,0]), "\t",
           "{:.5f}".format(src_loc.numpy()[0,0,1]), "\t",
           "{:.5f}".format(loss.numpy(),6), '\t',
           #"{:.5f}".format(loss_log10,6), '\t',
           "{:.5f}".format(med.numpy()[0]))
  if (i%1==0 and src_loc.shape[1]>1):
    print (i, src_loc.shape[1],
           "{:.5f}".format(loss.numpy(),6), '\t',
           "{:.5f}".format(med.numpy()[0]))

  ############################ Save Learning Data ############################
  #if dropped_ids.shape[0] > 0:
  src_ids, all_src_np = add_src_list(src_ids, drop_ids, all_src_np, src_loc.numpy())
  all_loss.append(loss.numpy())
  all_med.append(med.numpy()[0])
  all_iter.append(i)
  if(i==0):
    best_src.append(all_src_np[i].tolist())
    best_loss.append(loss.numpy())
    best_med.append(med.numpy()[0])
    best_iter.append(i)
  else:
    if loss.numpy() < min(best_loss):
      best_src.append(all_src_np[i].tolist())
      best_loss.append(loss.numpy())
      best_med.append(med.numpy()[0])
      best_iter.append(i)
  #############################################################################

  dx = tape.gradient(loss, src_loc)

  ######################## Apply Gradients  #########################
  d_move = loss / dx
  D = tf.sqrt(tf.square(d_move[:,:,0:1])+tf.square(d_move[:,:,1:2]))
  eta = tf.math.minimum(D,   max_step_size[:, :D.shape[1], :])
  eta = tf.math.maximum(eta, min_step_size[:, :D.shape[1], :])
  eps = tf.random.uniform(D.shape, minval=1, maxval=3, dtype=tf.float64)
  eta = eta / D * eps
  src_loc -= eta * d_move
  ###################################################################

  ########################### exclude false sources ###########################
  in_range = tf.math.logical_and(src_loc > -0.01, src_loc < 1.01)
  in_range = tf.math.logical_and(in_range[:,:,0], in_range[:,:,1])
  drop_ids = np.array([])
  if (in_range.shape[1] > 1):
    drop_ids = tf.reshape(tf.where(tf.logical_not(tf.reshape(in_range,[-1]))),[-1]).numpy()
    src_loc = tf.reshape(tf.gather_nd(src_loc, tf.where(in_range)), [1,-1,2])
    src_q = tf.ones([src_loc.shape[0],src_loc.shape[1],1], tf.float64) * (-1E-4)
  #############################################################################

all_src = all_src_np.tolist()
print ("True Source Location:", SRC_XY.numpy())
print ('The best source coordiante is ', best_src[-1], 'with loss of ', best_loss[-1], 'MED of ', best_med[-1])

### Post-Processing

#### Define Function - Save to Excel

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

import pandas as pd

def save_to_excel(filename, iter, loss, med, sources):
    data = {'Iter': iter, 'Loss': loss, 'MED': med}
    max_sources = max(len(source) for source in sources) // 2

    for i in range(max_sources):
        data[f'x{i+1}'] = []
        data[f'y{i+1}'] = []

    for index in range(len(iter)):
        for i in range(max_sources):
            if i < len(sources[index]) // 2:
                data[f'x{i+1}'].append(sources[index][2*i])
                data[f'y{i+1}'].append(sources[index][2*i+1])
            else:
                data[f'x{i+1}'].append(None)
                data[f'y{i+1}'].append(None)

    df = pd.DataFrame(data)
    df.to_excel(filename, index=False)

#### Save Predicted Path

In [ ]:
path = '/content/drive/My Drive/DPS/'
filename = path + 'Case1.xlsx'
save_to_excel(filename, best_iter, best_loss, best_med, best_src)